# Assignment 3: Building a Data Analysis Pipeline

The SQL code for the analysis problems






### Part 1. Basic analytics using BigQuery 

In [ ]:
 ## P1. Count the number of rows in the all_sessions_raw table (0 point)
 SELECT COUNT(*) FROM `data-to-insights.ecommerce.all_sessions_raw`

In [ ]:
## P2. Count the number of rows in the all_sessions table (0 point)
SELECT COUNT(*) FROM `data-to-insights.ecommerce.all_sessions`

In [ ]:
## P3. Check the min and max date of the records in the all_sessions (use the date column) (0 points).
SELECT MIN(date), MAX(date) FROM `data-to-insights.ecommerce.all_sessions` 

In [ ]:
## P4. Write a query that shows total unique visitors. (Your query determines the total views by counting product_views and the number of unique visitors by counting fullVisitorID) (5 points)
select
  COUNT(DISTINCT fullVisitorID) + COUNT(DISTINCT pageviews) as total_unique_visitor
from `data-to-insights.ecommerce.all_sessions`  where pageviews is not null; 

In [ ]:
## P5. Write a query that shows total unique visitors (fullVisitorID) by the referring site (channelGrouping) (5 points).
SELECT channelGrouping, COUNT(distinct fullVisitorID) as Unique_Visitors FROM `data-to-insights.ecommerce.all_sessions` group by channelGrouping ORDER BY Unique_Visitors;

In [ ]:
## P6. Write a query to list all the unique product names (v2ProductName) alphabetically (5 points).
SELECT DISTINCT(v2ProductName) FROM `data-to-insights.ecommerce.all_sessions`;

### Part 2. Duplicate detection in data (10%)

In [ ]:
## P7. Write a query that returns the number of duplicate records in the “all_sessions_raw” table. (Hint: you need to group the records by all the columns in the table) (5 points).
SELECT *,COUNT(*)
FROM `data-to-insights.ecommerce.all_sessions_raw`
GROUP BY fullVisitorId,channelGrouping,time,country,city, totaltransactionRevenue,transactions,timeOnSite, pageviews,sessionQualityDim,date,visitId,type,productRefundAmount,productQuantity,productPrice,productRevenue,productSKU,v2ProductName,v2ProductCategory,productVariant,currencyCode,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
HAVING COUNT(*) > 1 ORDER BY COUNT(*)deSC;

In [ ]:
## P8. Write a query that ensures there are no duplicate records in the “all_sessions” table (5 points).
SELECT *, Count(*) FROM `data-to-insights.ecommerce.all_sessions` 
group by fullVisitorId, channelGrouping, time, country, city, totalTransactionRevenue, transactions, timeOnSite, pageviews, sessionQualityDim, date,  visitId, type, productRefundAmount, productQuantity, productPrice, productRevenue, productSKU, v2ProductName, v2ProductCategory, productVariant, currencyCode, itemQuantity, itemRevenue, transactionRevenue, transactionId, pageTitle, searchKeyword, pagePathLevel1, eCommerceAction_type, eCommerceAction_step, eCommerceAction_option HAVING COUNT(*) > 1 ORDER BY COUNT(*)deSC;

### Part 3. Advanced analytics using BigQuery (25%)

In [ ]:
## P9. Write a query to list the five products with the most views (product_views) from all visitors (including people who have viewed the same product more than once). Your query counts the number of times a product (v2ProductName) was viewed (product_views), puts the list in descending order, and lists the top 5 entries (5 points).
select count(*) as product_views,v2ProductName AS ProductName from `data-to-insights.ecommerce.all_sessions` where type='PAGE' group by v2ProductName  order by product_views desc limit 5;

In [ ]:
## P10. Now refine the query to no longer double-count product views for visitors who have viewed a product many times. Each distinct product view should only count once per visitor (10 points).

#For finding distinct product viewed by each visitor
WITH individual_distinct_product_views AS (
SELECT fullVisitorId, v2ProductName 
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY fullVisitorId, v2ProductName )
 
#For collecting most viewed products and sort them
SELECT
  COUNT(*) AS total_product_views, v2ProductName 
FROM individual_distinct_product_views GROUP BY v2ProductName ORDER BY total_product_views DESC
LIMIT 5;


In [ ]:
## P11. expand your previous query to include the total number of distinct products ordered and the total number of total units ordered (productQuantity) (5 points).
SELECT
  COUNT(*) AS total_product_views,
  COUNT(Distinct(productQuantity)) AS  total_distinct_products_ordered,
  SUM(productQuantity) AS total_ordered_units,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions` WHERE type = 'PAGE' GROUP BY v2ProductName ORDER BY total_product_views DESC
LIMIT 5;

In [ ]:
## P12. Expand the query to include the average amount of product per order (total number of units ordered/total number of orders, or SUM(productQuantity)/COUNT(productQuantity)) (5 points).
SELECT
  COUNT(*) AS total_product_views,
  COUNT(productQuantity) AS ordered_units,
  SUM(productQuantity) AS Total_ordered_units,
  SUM(productQuantity) / COUNT(productQuantity) AS average_amount_of_product_per_order ,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions` WHERE type = 'PAGE' GROUP BY v2ProductName ORDER BY total_product_views DESC
LIMIT 5;

### Part 4. Schedule your analytics queries in ETL pipeline (20%)

In [ ]:
## P13. Build a pipeline that runs queries you wrote for P4, P5, and P12 (10 points). 

Explained in the Later cell (Part 4 and 6 Code)

## P14. Make sure the queries run in order, P4, then P5, then P15 (5 points) 

start >> bq_query4 >>bq_query5 >>bq_query12 >> table_export >> end

## P14. Schedule the pipeline so that it runs everyday. (5 points)

Explained in the Later cell (Part 4 and 6 Code)

### Part 5. Visualisation in Data Studio (20%)

In [1]:
## P15. Unique visitors by ID: Add the result of P4 as a number to your dashboard (5 points). 
## P16. Unique visitors by referring site: Add a bar chart that shows the result of the query you wrote in P5 (5 points). 
## P17. Highly viewed products: Add a pie chart that shows the result of P9 (5 points).
## P18. Details of highly viewed products: Add a table that shows the result of P12 in a table (5 points).

** Explained this parts in the video **

### Part 6. Exporting the results (10%)

In [ ]:
## P19. Using a command line operation, export the result of P12 to Google Cloud storage (5 points). 

## The command line code for exporting

bq extract --compression GZIP 'dataset_bigdata.datatable' gs://us-central1-assignment3bigd-d3a2ae5a-bucket/output/task6.csv

## P20. Add that step as the last step of the Airflow pipeline (5 points).


## Part 4 and 6 Code:
### The Python code of the ETL pipeline

In [ ]:
import datetime
import os
import logging
from airflow import DAG
from airflow import models
from airflow.contrib.operators.bigquery_operator import BigQueryOperator
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators import bash_operator
from google.cloud import bigquery #need to import 
from airflow.operators import bash_operator


today_date = datetime.datetime.now().strftime("%Y%m%d")
# table_name = 'omid.test_results' + '$' + today_date
table_name = 'dataset_bigdata.datatable'
yesterday = datetime.datetime.combine(
datetime.datetime.today() - datetime.timedelta(1),
datetime.datetime.min.time())
default_dag_args = {
# Setting start date as yesterday starts the DAG immediately when it is
# detected in the Cloud Storage bucket.
'start_date': yesterday,
# To email on failure or retry set 'email' arg to your email and enable
# emailing here.
'email_on_failure': False,
'email_on_retry': False,
# If a task fails, retry it once after waiting at least 5 minutes
'retries': 0,
'retry_delay': datetime.timedelta(minutes=5),
'project_id': models.Variable.get('gcp_project')
}
with DAG(dag_id='bigquery-dag-file',


# Continue to run DAG once per day
               ## P14. Schedule the pipeline so that it runs everyday. (5 points) ##

schedule_interval=datetime.timedelta(days=1),
default_args=default_dag_args) as dag:

	start = DummyOperator(task_id='start')
	end = DummyOperator(task_id='end')
	logging.info('trying to bq_query: ')
	logging.info('table name: ' + table_name)

	sql1 = f""" select
  COUNT(DISTINCT fullVisitorID) + COUNT(DISTINCT pageviews) as total_unique_visitor
from `data-to-insights.ecommerce.all_sessions`  where pageviews is not null
	"""
	bq_query4 = BigQueryOperator(
	task_id='big_query4',
	sql=sql1,
	destination_dataset_table=table_name,
	gcp_conn_id='bigquery_default',
	use_legacy_sql=False,
	write_disposition='WRITE_TRUNCATE',
	create_disposition='CREATE_IF_NEEDED',
	dag=dag)

	sql2 = f""" SELECT channelGrouping, COUNT(distinct fullVisitorID) as Unique_Visitors FROM `data-to-insights.ecommerce.all_sessions` group by channelGrouping ORDER BY Unique_Visitors
	"""
	bq_query5 = BigQueryOperator(
	task_id='big_query5',
	sql=sql2,
	destination_dataset_table=table_name,
	gcp_conn_id='bigquery_default',
	use_legacy_sql=False,
	write_disposition='WRITE_TRUNCATE',
	create_disposition='CREATE_IF_NEEDED',
	dag=dag)

	sql3 = f""" SELECT
  COUNT(*) AS total_product_views,
  COUNT(productQuantity) AS ordered_units,
  SUM(productQuantity) AS Total_ordered_units,
  SUM(productQuantity) / COUNT(productQuantity) AS average_amount_of_product_per_order ,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions` WHERE type = 'PAGE' GROUP BY v2ProductName ORDER BY total_product_views DESC
LIMIT 5;

	"""
	bq_query12 = BigQueryOperator(
	task_id='big_query12',
	sql=sql3,
	destination_dataset_table=table_name,
	gcp_conn_id='bigquery_default',
	use_legacy_sql=False,
	write_disposition='WRITE_TRUNCATE',
	create_disposition='CREATE_IF_NEEDED',
	dag=dag)
             ### P20. Add that step as the last step of the Airflow pipeline (5 points).  ###

	table_export=bash_operator.BashOperator(
		task_id="export",
		bash_command="bq extract --compression GZIP 'dataset_bigdata.datatable' gs://us-central1-assignment3bigd-d3a2ae5a-bucket/output/task6.csv" )

            ##  P14. Make sure the queries run in order, P4, then P5, then P15 (5 points) ##   
start >> bq_query4 >>bq_query5 >>bq_query12 >> table_export >> end

## A link to the recorded video as instructed in the description 
### Youtube Link : https://youtu.be/wHF8IzOa8PI 